### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])



school_data_complete.to_csv("output.csv")

## Local Government Area Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average maths score 

* Calculate the average reading score

* Calculate the percentage of students with a passing maths score (50 or greater)

* Calculate the percentage of students with a passing reading score (50 or greater)

* Calculate the percentage of students who passed maths **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#calc total schools
num_schools = school_data_complete['school_name'].nunique()
#calc total number of students
num_students = school_data_complete['Student ID'].nunique()
#calc total budget
total_budget = school_data['budget'].sum()

In [3]:
#Calc average math score
avg_math = school_data_complete['maths_score'].mean()
#Calc average reading score
avg_reading = school_data_complete['reading_score'].mean()

In [4]:
#set passing score
pass_score = 50
#calc percentage of students passing math
pass_math = ((student_data['maths_score'] >= pass_score).sum()) / num_students
#calc percentage of students passing math
pass_reading = ((student_data['reading_score'] >= pass_score).sum()) / num_students

#calc percentage of students passing math & Reading
pass_reading_maths = (((student_data['reading_score'] >= pass_score) & (student_data['maths_score'] >= pass_score)).sum()) / num_students

In [5]:
#Create DF

area_summary = pd.DataFrame({
                "Total Schools": num_schools,
                "Total Students": num_students,
                "Total Budget": total_budget,
                "Average Math Score": avg_math,
                "Average Reading Score": avg_reading,
                "% Passing Maths": pass_math * 100,
                "% Passing Reading": pass_reading * 100,
                "% Overall Passing": pass_reading_maths * 100}
                ,index=[0]) #Need to set an index as all values are singlular

In [6]:
#tidy table output using mapping
area_summary['Total Students'] = area_summary['Total Students'].map("{:,}".format)
area_summary['Total Budget'] = area_summary['Total Budget'].map("${:,}".format)
area_summary['Average Math Score'] = area_summary['Average Math Score'].map("{:.2f}".format)
area_summary['Average Reading Score'] = area_summary['Average Reading Score'].map("{:.2f}".format)
area_summary['% Passing Maths'] = (area_summary['% Passing Maths']).map("{:.2f}%".format)
area_summary['% Passing Reading'] = (area_summary['% Passing Reading']).map("{:.2f}%".format)
area_summary['% Overall Passing'] = (area_summary['% Overall Passing']).map("{:.2f}%".format)
area_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428",70.34,69.98,86.08%,84.43%,72.81%


## School Summary

* Create an overview table that summarises key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed maths **and** reading.)
  
* Create a dataframe to hold the above results

In [7]:
# add extra column to school_data_complete for whether student is passing math:
school_data_complete['pass_math'] = school_data_complete['maths_score'] >= pass_score
# add extra column to school_data_complete for whether student is passing reading:
school_data_complete['pass_reading'] = school_data_complete['reading_score'] >= pass_score
# add extra column to school_data_complete for whether student is passing math and reading: 
school_data_complete['pass_reading_math'] = (school_data_complete['pass_math'] & school_data_complete['pass_reading'])

In [8]:
# groupby school and calculate total number of students, size and budget
school_summary = school_data_complete.groupby(['school_name', 'type']).agg({'Student ID': 'count', #total student number
                                                                           'budget': 'mean', #budget per school
                                                                           'reading_score' : 'mean', #avg reading score
                                                                           'maths_score' : 'mean', #avg math score
                                                                            'pass_math': 'sum', #num students pass math
                                                                            'pass_reading': 'sum', #num students pass reading
                                                                            'pass_reading_math': 'sum' #num students pass both
                                                                            })

#Reset indexes:
school_summary = school_summary.reset_index()


In [9]:
#add column for per student budget
school_summary['Per Student Budget'] = school_summary['budget'] / school_summary['Student ID']

#add column for % Passing Maths
school_summary['% Passing Maths'] = round((school_summary['pass_math'] / school_summary['Student ID']) * 100,2)

#add column for Passing Reading
school_summary['% Passing Reading'] = round((school_summary['pass_reading'] / school_summary['Student ID']) * 100,2)

#add column for Overall Passing (The percentage of students that passed maths **and** reading.)
school_summary['% Overall Passing'] = round((school_summary['pass_reading_math'] / school_summary['Student ID']) * 100,2)

#Round reading_score
school_summary['reading_score'] = round(school_summary['reading_score'],2)

#Round maths_score
school_summary['maths_score'] = round(school_summary['maths_score'],2)



In [10]:
#Select only the needed columns
per_school_summary = school_summary.drop(['pass_math', 'pass_reading', 'pass_reading_math'], axis=1)
#put the summary into another DF for use in later steps
per_school_summary_raw = per_school_summary


#Rename columns
per_school_summary = per_school_summary.rename({'school_name': "School Name",
                             'type': 'School Type',
                             'reading_score': 'Average Reading Score',
                             'maths_score': 'Average Maths Score',
                             'budget': 'Total School Budget',
                             'Student ID': 'Total Students'
                             }, axis = 1)



#Make the formatting nice
per_school_summary['Total School Budget'] = per_school_summary['Total School Budget'].map("${:,.2f}".format)
per_school_summary['Per Student Budget'] = per_school_summary['Per Student Budget'].map("${:,.2f}".format)
per_school_summary['% Passing Maths'] = (per_school_summary['% Passing Maths']).map("{:.2f}%".format)
per_school_summary['% Passing Reading'] = (per_school_summary['% Passing Reading']).map("{:.2f}%".format)
per_school_summary['% Overall Passing'] = (per_school_summary['% Overall Passing']).map("{:.2f}%".format)

#reorder the columns to put budget columns together
new_order = ['School Name','School Type','Total School Budget','Per Student Budget','Average Reading Score','Average Maths Score','% Passing Maths','% Passing Reading', '% Overall Passing']
per_school_summary = per_school_summary.reindex(columns=new_order)

#Display the DF
per_school_summary.head(15)


,School Name,School Type,Total School Budget,Per Student Budget,Average Reading Score,Average Maths Score,% Passing Maths,% Passing Reading,% Overall Passing
0,Bailey High School,Government,"$3,124,928.00",$628.00,71.01,72.35,91.64%,87.38%,80.08%
1,Cabrera High School,Independent,"$1,081,356.00",$582.00,71.36,71.66,90.85%,89.07%,80.79%
2,Figueroa High School,Government,"$1,884,411.00",$639.00,69.08,68.70,81.65%,82.81%,67.65%
3,Ford High School,Government,"$1,763,916.00",$644.00,69.57,69.09,82.44%,82.22%,67.47%
4,Griffin High School,Independent,"$917,500.00",$625.00,71.25,71.79,91.21%,88.49%,81.34%
5,Hernandez High School,Government,"$3,022,020.00",$652.00,69.19,68.87,80.95%,81.88%,66.36%
6,Holden High School,Independent,"$248,087.00",$581.00,71.66,72.58,89.93%,88.52%,78.92%
7,Huang High School,Government,"$1,910,635.00",$655.00,68.91,68.94,81.69%,81.45%,66.71%
8,Johnson High School,Government,"$3,094,650.00",$650.00,69.04,68.84,82.06%,81.98%,67.19%
9,Pena High School,Independent,"$585,858.00",$609.00,71.61,72.09,91.68%,86.59%,79.21%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [11]:
#Select top 5 school. nlargest automaticly sorts
top_five_schools = per_school_summary_raw.nlargest(5, '% Overall Passing')


#Rename columns
top_five_schools = top_five_schools.rename({'school_name': "School Name",
                             'type': 'School Type',
                             'reading_score': 'Average Reading Score',
                             'maths_score': 'Average Maths Score',
                             'budget': 'Total School Budget',
                             'Student ID': 'Total Students'
                             }, axis = 1)



#Make the formatting nice
top_five_schools['Total School Budget'] = top_five_schools['Total School Budget'].map("${:,.2f}".format)
top_five_schools['Per Student Budget'] = top_five_schools['Per Student Budget'].map("${:,.2f}".format)
top_five_schools['% Passing Maths'] = (top_five_schools['% Passing Maths']).map("{:.2f}%".format)
top_five_schools['% Passing Reading'] = (top_five_schools['% Passing Reading']).map("{:.2f}%".format)
top_five_schools['% Overall Passing'] = (top_five_schools['% Overall Passing']).map("{:.2f}%".format)

#reorder the columns to put budget columns together
new_order = ['School Name','School Type','Total School Budget','Per Student Budget','Average Reading Score','Average Maths Score','% Passing Maths','% Passing Reading', '% Overall Passing']
top_five_schools =top_five_schools.reindex(columns=new_order)

#Display
top_five_schools






,School Name,School Type,Total School Budget,Per Student Budget,Average Reading Score,Average Maths Score,% Passing Maths,% Passing Reading,% Overall Passing
4,Griffin High School,Independent,"$917,500.00",$625.00,71.25,71.79,91.21%,88.49%,81.34%
1,Cabrera High School,Independent,"$1,081,356.00",$582.00,71.36,71.66,90.85%,89.07%,80.79%
0,Bailey High School,Government,"$3,124,928.00",$628.00,71.01,72.35,91.64%,87.38%,80.08%
14,Wright High School,Independent,"$1,049,400.00",$583.00,70.97,72.05,91.78%,86.67%,79.72%
10,Rodriguez High School,Government,"$2,547,363.00",$637.00,70.94,72.05,90.80%,87.40%,79.42%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [12]:
#Select top 5 school. nlargest automaticly sorts
bottom_five_schools = per_school_summary_raw.nsmallest(5, '% Overall Passing')

#Rename columns
bottom_five_schools = bottom_five_schools.rename({'school_name': "School Name",
                             'type': 'School Type',
                             'reading_score': 'Average Reading Score',
                             'maths_score': 'Average Maths Score',
                             'budget': 'Total School Budget',
                             'Student ID': 'Total Students'
                             }, axis = 1)



#Make the formatting nice
bottom_five_schools['Total School Budget'] = bottom_five_schools['Total School Budget'].map("${:,.2f}".format)
bottom_five_schools['Per Student Budget'] = bottom_five_schools['Per Student Budget'].map("${:,.2f}".format)
bottom_five_schools['% Passing Maths'] = (bottom_five_schools['% Passing Maths']).map("{:.2f}%".format)
bottom_five_schools['% Passing Reading'] = (bottom_five_schools['% Passing Reading']).map("{:.2f}%".format)
bottom_five_schools['% Overall Passing'] = (bottom_five_schools['% Overall Passing']).map("{:.2f}%".format)

#reorder the columns to put budget columns together
new_order = ['School Name','School Type','Total School Budget','Per Student Budget','Average Reading Score','Average Maths Score','% Passing Maths','% Passing Reading', '% Overall Passing']
bottom_five_schools =bottom_five_schools.reindex(columns=new_order)

#Display
bottom_five_schools

,School Name,School Type,Total School Budget,Per Student Budget,Average Reading Score,Average Maths Score,% Passing Maths,% Passing Reading,% Overall Passing
5,Hernandez High School,Government,"$3,022,020.00",$652.00,69.19,68.87,80.95%,81.88%,66.36%
7,Huang High School,Government,"$1,910,635.00",$655.00,68.91,68.94,81.69%,81.45%,66.71%
8,Johnson High School,Government,"$3,094,650.00",$650.00,69.04,68.84,82.06%,81.98%,67.19%
13,Wilson High School,Independent,"$1,319,574.00",$578.00,68.88,69.17,82.79%,81.30%,67.46%
3,Ford High School,Government,"$1,763,916.00",$644.00,69.57,69.09,82.44%,82.22%,67.47%


## Maths Scores by Year

* Create a table that lists the average maths score for students of each year level (9, 10, 11, 12) at each school.

  * Create a pandas series for each year. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [13]:
#groupby to create df of school math average scores by year
by_year_maths_avg = school_data_complete.groupby(['year', 'school_name'])['maths_score'].mean()

In [14]:
#Pull out each year using loc and index (grade number)
year9_math = by_year_maths_avg.loc[9]
year10_math = by_year_maths_avg.loc[10]
year11_math = by_year_maths_avg.loc[11]
year12_math = by_year_maths_avg.loc[12]

In [15]:
#calc mean math by school
year9_math_by_school = year9_math.groupby('school_name').mean()
year10_math_by_school = year10_math.groupby('school_name').mean()
year11_math_by_school = year11_math.groupby('school_name').mean()
year12_math_by_school = year12_math.groupby('school_name').mean()

In [16]:
#create df from the year series, round to tidy table 
yearly_math_by_school = pd.DataFrame({'Year 9': round(year9_math_by_school,2),
                                     'Year 10': round(year10_math_by_school,2),
                                     'Year 11': round(year11_math_by_school,2),
                                     'Year 12': round(year12_math_by_school,2)})

#Reset index
yearly_math_by_school.reset_index(inplace=True)

yearly_math_by_school.head(15)

,school_name,Year 9,Year 10,Year 11,Year 12
0,Bailey High School,72.49,71.90,72.37,72.68
1,Cabrera High School,72.32,72.44,71.01,70.60
2,Figueroa High School,68.48,68.33,68.81,69.33
3,Ford High School,69.02,69.39,69.25,68.62
4,Griffin High School,72.79,71.09,71.69,71.47
5,Hernandez High School,68.59,68.87,69.15,68.99
6,Holden High School,70.54,75.11,71.64,73.41
7,Huang High School,69.08,68.53,69.43,68.64
8,Johnson High School,69.47,67.99,68.64,69.29
9,Pena High School,72.00,72.40,72.52,71.19


## Reading Score by Year

* Perform the same operations as above for reading scores

In [17]:
#groupby to create df of school reading average scores by year
by_year_reading_avg = school_data_complete.groupby(['year', 'school_name'])['reading_score'].mean()

In [18]:
#Pull out each year using loc and index (grade number)
year9_reading = by_year_reading_avg.loc[9]
year10_reading = by_year_reading_avg.loc[10]
year11_reading = by_year_reading_avg.loc[11]
year12_reading = by_year_reading_avg.loc[12]

In [19]:
#calc mean reading by school
year9_reading_by_school = year9_reading.groupby('school_name').mean()
year10_reading_by_school = year10_reading.groupby('school_name').mean()
year11_reading_by_school = year11_reading.groupby('school_name').mean()
year12_reading_by_school = year12_reading.groupby('school_name').mean()

In [20]:
#create df from the year series 
yearly_reading_by_school = pd.DataFrame({'year 9': year9_reading_by_school,
                                     'year 10': year10_reading_by_school,
                                     'year 11': year10_reading_by_school,
                                     'year 12': year12_reading_by_school})
yearly_reading_by_school

,year 9,year 10,year 11,year 12
school_name,,,,
Bailey High School,70.901920,70.848265,70.848265,72.195525
Cabrera High School,71.172348,71.328326,71.328326,71.856021
Figueroa High School,70.261682,67.677588,67.677588,69.082126
Ford High School,69.615846,68.988701,68.988701,68.849722
Griffin High School,72.026895,70.746305,70.746305,69.434932
Hernandez High School,68.477569,70.621842,70.621842,69.244136
Holden High School,71.598425,71.096491,71.096491,70.481928
Huang High School,68.670616,69.516297,69.516297,68.671795
Johnson High School,68.719286,69.295029,69.295029,67.992521


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [21]:
#Initiate bin sizes & Labels
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

#Assign summary to new df
school_spending_df = per_school_summary_raw

school_spending_df = school_spending_df.rename({'school_name': "School Name",
                             'type': 'School Type',
                             'reading_score': 'Average Reading Score',
                             'maths_score': 'Average Maths Score',
                             'budget': 'Total School Budget',
                             'Student ID': 'Total Students'
                             }, axis = 1)

#use pd.cut() to assign bins
school_spending_df['Spending Ranges (Per Student)'] = pd.cut(school_spending_df['Per Student Budget'], bins=spending_bins, labels=labels)

In [22]:
#Create series with avg scores and passing scores
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Maths Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Maths"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()



In [23]:
spending_summary = pd.DataFrame({"Average maths score": round(spending_math_scores,2),
                                 "Average reading score": round(spending_reading_scores,2),
                                 "% passing maths": round(spending_passing_math,2),
                                 "% passing reading": round(spending_passing_reading,2),
                                 "% overall passing": round(overall_passing_spending,2)})
spending_summary

,Average maths score,Average reading score,% passing maths,% passing reading,% overall passing
Spending Ranges (Per Student),,,,,
<$585,71.36,70.72,88.84,86.39,76.72
$585-630,72.06,71.03,91.52,87.29,79.88
$630-645,69.86,69.84,84.68,83.76,71.00
$645-680,68.88,69.05,81.57,81.77,66.75


## Scores by School Size

* Perform the same operations as above, based on school size.

In [24]:
#Initiate bin sizes & Labels
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [25]:
size_df = per_school_summary_raw

#Use pd.cut to assign bins
size_df['School Size'] = pd.cut(size_df['Student ID'], bins=size_bins, labels=labels)


In [26]:
#Create series with avg scores and passing scores
size_math_scores = size_df.groupby(["School Size"])["maths_score"].mean()
size_reading_scores = size_df.groupby(["School Size"])["reading_score"].mean()
size_passing_math = size_df.groupby(["School Size"])["% Passing Maths"].mean()
size_passing_reading = size_df.groupby(["School Size"])["% Passing Reading"].mean()
overall_passing_size = size_df.groupby(["School Size"])["% Overall Passing"].mean()

In [27]:
#Create df from scores
size_summary = pd.DataFrame({"Average maths score": round(size_math_scores,2),
                                 "Average reading score": round(size_reading_scores,2),
                                 "% passing maths": round(size_passing_math,2),
                                 "% passing reading": round(size_passing_reading,2),
                                 "% overall passing": round(overall_passing_size, 2)})
size_summary

,Average maths score,Average reading score,% passing maths,% passing reading,% overall passing
School Size,,,,,
Small (<1000),72.34,71.63,90.80,87.56,79.06
Medium (1000-2000),71.42,70.72,89.85,86.71,78.04
Large (2000-5000),69.75,69.58,84.25,83.30,70.29


## Scores by School Type

* Perform the same operations as above, based on school type

In [28]:
#Create series with avg scores and passing scores
type_math_scores = per_school_summary_raw.groupby(["type"])["maths_score"].mean()
type_reading_scores = per_school_summary_raw.groupby(["type"])["reading_score"].mean()
type_passing_math = per_school_summary_raw.groupby(["type"])["% Passing Maths"].mean()
type_passing_reading = per_school_summary_raw.groupby(["type"])["% Passing Reading"].mean()
overall_passing_type = per_school_summary_raw.groupby(["type"])["% Overall Passing"].mean()

In [29]:
#Create df from scores
type_summary = pd.DataFrame({"Average maths score": round(type_math_scores,2),
                                 "Average reading score":round(type_reading_scores,2),
                                 "% passing maths": round(type_passing_math,2),
                                 "% passing reading": round(type_passing_reading,2),
                                 "% overall passing": round(overall_passing_type,2)})
type_summary

,Average maths score,Average reading score,% passing maths,% passing reading,% overall passing
type,,,,,
Government,69.83,69.68,84.46,83.59,70.70
Independent,71.37,70.72,89.20,86.25,76.97
